# <b style="color: #151151">Lab 2 - Arend Wong (aw3146) - 2021-05-21<b>

<i style='color: blue;'>
Due Monday by 11:59pm Points 100 Submitting a website url or a file upload
Lab #2

1. Recode 2 different variables into new categories. They can both be continuous-ish or both be nominal-ish, or one of each. Tell me what you did and explain the variable(s).

2. Use one (or both) of your recoded variables to do a cross-tabulation. Explain your results.

3. Run a linear regression with 1 independent and 1 dependent variable; make all of the recodes necessary to make the model as easy to interpret as possible; and explain your results.

4. Plot two variables, either as a scatter plot or boxplot; add in trend/regression lines; and explain your results.
</i>


In [6]:
import pandas as pd
import numpy as np
import re

In [4]:
# Read CSV into a dataframe
url = "https://raw.githubusercontent.com/the-pudding/data/master/pockets/measurements.csv"
